In [1]:
# Standard imports wo work with data, graphs, APIs, etc.

# Compatibility with Python 3
from __future__ import print_function, division

import pandas as pd
import geopandas as gpd
import numpy as np
import json
import requests
import urllib2
import urllib
import ast
from cStringIO import StringIO
from pandas.tools.plotting import scatter_matrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
from scipy.interpolate import *
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
import zipfile
from geopandas import GeoDataFrame
import pysal as ps
from fiona.crs import from_epsg
import matplotlib.colors as mcolors
import operator
from matplotlib.colors import ListedColormap    
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pylab as plt
import seaborn as sns
import csv
import dateutil
import os
import psycopg2

%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/matplotlib/__init__.py:878: UserWarning: svg.embed_char_paths is deprecated and replaced with svg.fonttype; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['griddata', 'insert', 'polyint']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
## read json file for password 
import json
json_data = open("password.json").read() #reading api key from api.json
passw = json.loads(json_data)

PASSWORD = str(passw["PASSWORD"])

In [ ]:
# https://support.dominodatalab.com/hc/en-us/articles/207346026-Connecting-to-Redshift
# OAG Redshift Reference:
# https://docs.google.com/document/d/1FsTqbR9tk0Z7vxbDJ1vaNiEHfKEinxtOTU8YvBnUNI8/edit

HOST = "research1.cwcou8cgfav2.us-east-1.redshift.amazonaws.com"
PORT = 5439 # redshift default
USER = "cusp2017"

# Password either from environmental variable or input raw
# PASSWORD = os.environ['REDSHIFT_PASSWD']
#PASSWORD = raw_input()

# oag_civis
DATABASE = 'oag_civis'

def db_connection():
    conn = psycopg2.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database=DATABASE,
    )
    return conn


In [23]:
# agn_buildings
example_query = "select distinct(tablename) from pg_table_def where schemaname = 'agn_buildings';"

conn = db_connection()
try:
    cursor = conn.cursor()
    cursor.execute(example_query)
    results = cursor.fetchall() # careful, the results could be huge
    conn.commit()
    print (results)
finally:
    conn.close()

# using pandas
import pandas as pd
conn = db_connection()
try:
    df = pd.read_sql(example_query, conn)
    #df.to_csv('results/outfile.csv', index=False)
finally:
    conn.close()

[]


In [ ]:
# agn_buildings - 311 complaints

311_query = "select top 10 * from agn_buildings.complaints_311 limit 10;"

conn =db_connection()
try:
    df = pd.read_sql(311_query, conn)
finally:
    conn.close()
    
df.head(10)

In [ ]:
## agn_buildings - dob_complaints, dob_violations, dob_permits, hpd_complaints, hpd_violations, hpd_litigation

sample_query = "select * from agn_buildings.dob_complaints, agn_buildings.dob_violations limit 10;"
try:
    df = pd.read_sql(sample_query, conn)
finally:
    conn.close()
    
df.head(10)

In [24]:
example_query = "Select * from agn_buildings.pluto limit 10;"
# select top 100 * from [buildings_and_violations].[dbo].[OATH_ECB_Violations]

conn = db_connection()
try:
    df = pd.read_sql(example_query, conn)
finally:
    conn.close()
df.head()

,borough,block,lot,cd,ct2010,cb2010,schooldist,council,zipcode,firecomp,...,sanborn,taxmap,edesignum,appbbl,appdate,plutomapid,version,latitude,longitude,bbldash
0,BK,1,50,302,21.0,2000.0,13.0,33.0,11201.0,L118,...,302 007,30101.0,E-231,0.0,None,1,16V2,40.704621,-73.987059,3-1-50
1,BK,3,1,302,21.0,3002.0,13.0,33.0,11201.0,L118,...,302 007,30101.0,None,0.0,None,1,16V2,40.705065,-73.984426,3-3-1
2,BK,3,35,302,21.0,NaN,13.0,33.0,11201.0,L118,...,302 007,30101.0,None,0.0,None,2,16V2,NaN,NaN,3-3-35
3,BK,5,8,302,21.0,NaN,13.0,33.0,11201.0,L118,...,300 000,30101.0,None,0.0,None,4,16V2,NaN,NaN,3-5-8
4,BK,6,10,302,21.0,NaN,13.0,33.0,11201.0,E207,...,302 009,30101.0,None,0.0,None,1,16V2,NaN,NaN,3-6-10


In [29]:
example_query = "Select * from agn_buildings.acris_rp_parties where name ilike '%CIAMPA%';"
# select top 100 * from [buildings_and_violations].[dbo].[OATH_ECB_Violations]

conn = db_connection()
try:
    df = pd.read_sql(example_query, conn)
finally:
    conn.close()

In [30]:
df.head()

,document_id,record_type,party_type,name,address_1,address_2,country,city,state,zip,good_through_date
0,FT_4720007086472,P,1,CIAMPA NORTH CO,None,None,None,None,None,None,07/31/2015 12:00:00 AM
1,BK_7420025701910,P,1,CIAMPA BRADDOCK INC,None,None,None,None,None,None,07/31/2015 12:00:00 AM
2,BK_7640095201154,P,1,CIAMPA BENJAMIN,None,None,None,None,None,None,07/31/2015 12:00:00 AM
3,2013040301537001,P,1,CIAMPA NORTH LLC,136-26 37TH AVENUE,None,US,FLUSHING,NY,11354,07/31/2015 12:00:00 AM
4,BK_7520027100185,P,1,CIAMPA JOSEPH D JR,None,None,None,None,None,None,07/31/2015 12:00:00 AM


In [3]:
# This snippet mocks up the desired interaction with the search box 
# from the main form
# It will take a list of terms from the raw input (separated by comas)
# And build a SQL query that does a SELECT query with ilikes and ANDs

queryInput = raw_input()
tokens = queryInput.split(',')
SQLWhere = ""

for token in tokens:
    SQLWhere = SQLWhere + " name ilike '%" + token + "%' AND"

SQLWhere = SQLWhere[:-3] + ";"
    
example_query = '''
Select * from agn_buildings.acris_rp_parties
where'''
example_query = example_query + SQLWhere

print (example_query)

ciampa,crescent

Select * from agn_buildings.acris_rp_parties
where name ilike '%ciampa%' AND name ilike '%crescent%' ;


In [5]:
# This cell executes the query built on the previous cell
# and display the top five rows

conn = db_connection()
try:
    df = pd.read_sql(example_query, conn)
finally:
    conn.close()

df.head()

,document_id,record_type,party_type,name,address_1,address_2,country,city,state,zip,good_through_date
0,2016030701225001,P,1,CIAMPA CRESCENT LLC,C/O CIAMPA ORGANIZATION,"241-02 NORTHERN BOULEVARD, 3RD FOOR",US,DOUGLASTON,NY,11362,03/31/2016 12:00:00 AM
1,2013022500988002,P,1,CIAMPA CRESCENT LLC,C/O CIAMPA ORGANIZATION,136-26 37TH AVENUE,US,FLUSHING,NY,11354,07/31/2015 12:00:00 AM
2,2009060100763006,P,1,CIAMPA CRESCENT LLC,None,None,None,None,None,None,07/31/2015 12:00:00 AM
3,2008011001216001,P,1,CIAMPA CRESCENT LLC,C/O THE CIAMPA ORGANIZATION,136-26 37TH AVENUE,US,FLUSHING,NY,11354,07/31/2015 12:00:00 AM
4,2009060100763002,P,1,CIAMPA CRESCENT LLC,C/O THE CIAMPA ORGANIZATION,136-26 37TH AVENUE,US,FLUSHING,NY,11354,07/31/2015 12:00:00 AM


In [6]:
# Query the tables on the catalog

example_query = '''
set search_path to agn_buildings;
select distinct(id) table_id
,trim(datname)   db_name
,trim(nspname)   schema_name
,trim(relname)   table_name
from stv_tbl_perm
join pg_class on pg_class.oid = stv_tbl_perm.id
join pg_namespace on pg_namespace.oid = relnamespace
join pg_database on pg_database.oid = stv_tbl_perm.db_id;
'''

conn = db_connection()
try:
    df = pd.read_sql(example_query, conn)
    #df.to_csv('results/outfile.csv', index=False)
finally:
    conn.close()
df

,table_id,db_name,schema_name,table_name
0,148770,oag_civis,logs,stl_load_errors
1,148774,oag_civis,logs,stl_query
2,148783,oag_civis,logs,stl_stream_segs
3,148791,oag_civis,logs,stl_undone
4,1397449,oag_civis,crim_cefc,ghosttown_whiteboard_ppresent_trades
5,304720,oag_civis,econ_housing,dfs_test
6,452284,oag_civis,econ_housing,deedtheft_geocode_input
7,681479,oag_civis,econ_housing,deedtheft_geocode_joined
8,452274,oag_civis,econ_housing,deedtheft_lawyerscmtecases_cassed
9,392700,oag_civis,econ_housing,deedtheft_lawyerscmtecases_final
